<a href="https://colab.research.google.com/github/yoseforaz0990/ML-templates/blob/main/deep_learning/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Step Name                                | Description                                                                                                             |
|------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
| Importing necessary libraries           | Importing the required libraries for building the ResNet model.                                                        |
| Defining the Residual Block             | Defining a single residual block with skip connection.                                                                  |
| Creating the ResNet model               | Building the ResNet model using the defined residual blocks.                                                           |
| Compiling the ResNet model              | Compiling the ResNet model with categorical cross-entropy loss and Adam optimizer.                                     |
| Printing the ResNet model summary       | Displaying the summary of the ResNet model architecture.                                                               |


In [ ]:
# Step 1: Importing necessary libraries
# Description: Importing the required libraries for building the ResNet model.

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

# Step 2: Defining the Residual Block
# Description: Defining a single residual block with skip connection.

def residual_block(x, filters, strides=(1, 1), activation="relu"):
    identity = x

    x = Conv2D(filters, kernel_size=(3, 3), strides=strides, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)

    x = Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
    x = BatchNormalization()(x)

    if strides != (1, 1) or identity.shape[-1] != filters:
        identity = Conv2D(filters, kernel_size=(1, 1), strides=strides, padding="same")(identity)
        identity = BatchNormalization()(identity)

    x = Add()([x, identity])
    x = Activation(activation)(x)
    return x

# Step 3: Creating the ResNet model
# Description: Building the ResNet model using the defined residual blocks.

def create_resnet(input_shape=(32, 32, 3), num_classes=10, num_filters=16, num_blocks_list=[2, 2, 2]):
    inputs = Input(shape=input_shape)

    x = Conv2D(num_filters, kernel_size=(3, 3), strides=(1, 1), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    for num_blocks in num_blocks_list:
        for i in range(num_blocks):
            if i == 0:
                x = residual_block(x, num_filters, strides=(2, 2), activation="relu")
            else:
                x = residual_block(x, num_filters, strides=(1, 1), activation="relu")
        num_filters *= 2

    x = AveragePooling2D(pool_size=(4, 4))(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Step 4: Compiling the ResNet model
# Description: Compiling the ResNet model with categorical cross-entropy loss and Adam optimizer.

model = create_resnet()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Step 5: Printing the ResNet model summary
# Description: Displaying the summary of the ResNet model architecture.

model.summary()
